In [104]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.linalg import svd
from math import *

# Сингулярное разложение

В этом ДЗ есть четыре варианта заданий, с разной разбалловкой. Вы можете сделать одно из них и получить соответствующее количество баллов (из 10):  

- 4 - пакетная функция SVD с обычной матрицей
- 7 - собственное SVD с обычной матрицей
- 8 - SVD с реальным датасетом, пакетная реализация
- 10 - реальный датасет, собственная реализация

## (4-7) Типовая матрица

Постройте сингулярное разложение для матрицы A:

$$A=\left(
\begin{array}{rrr}
1 & -1 & -2 \\
-7/3 &  1/3 & 2/3\\
1/3 & -7/3 & -2/3 \\
-5/3 & 5/3 & -2/3
\end{array}
\right) \, $$

Составьте матрицы $U$, $\Sigma$, $V$ (в неусеченном виде)

При написании собственной реализации можно использовать *np.linalg.solve* и *np.linalg.inv*

In [98]:
A = np.array([[1, -1, -2],
              [-7/3, 1/3, 2/3],
              [1/3, -7/3, -2/3],
              [-5/3, 5/3, -2/3]])

print(A)

[[ 1.         -1.         -2.        ]
 [-2.33333333  0.33333333  0.66666667]
 [ 0.33333333 -2.33333333 -0.66666667]
 [-1.66666667  1.66666667 -0.66666667]]


In [185]:
### svd() ###

def do_svd(mat):
    U, Sigma, VT = svd(mat)
    U = pd.DataFrame(U)
    VT = pd.DataFrame(VT)
    Sigma = pd.DataFrame(Sigma)
    return U, Sigma, VT

In [186]:
U = do_svd(A)[0]

Sigmas = do_svd(A)[1].to_numpy().T

VT = do_svd(A)[2]

print('U:')
print(U,'\n\n')
print('Sigmas:')
print(Sigmas,'\n\n')
print('V^T:\n',VT,'\n')


Sigma = np.eye(4,3) * Sigmas

print(Sigma,'\n')

print('Check tre Result of (U * Sigma * VT) :')
print(U @ Sigma @ VT)

U:
     0         1         2    3
0 -0.5  0.223607  0.670820 -0.5
1  0.5  0.670820 -0.223607 -0.5
2 -0.5  0.670820 -0.223607  0.5
3  0.5  0.223607  0.670820  0.5 


Sigmas:
[[4. 2. 2.]] 


V^T:
           0         1         2
0 -0.666667  0.666667  0.333333
1 -0.745356 -0.596285 -0.298142
2 -0.000000  0.447214 -0.894427 

[[4. 0. 0.]
 [0. 2. 0.]
 [0. 0. 2.]
 [0. 0. 0.]] 

Check tre Result of (U * Sigma * VT) :
          0         1         2
0  1.000000 -1.000000 -2.000000
1 -2.333333  0.333333  0.666667
2  0.333333 -2.333333 -0.666667
3 -1.666667  1.666667 -0.666667


## (8-10) Работа с датасетом

Будем использовать [MovieLens 100k](http://grouplens.org/datasets/movielens/) (*MovieLens Latest Datasets (small)*, ratings.csv)

Вам нужно:
- скачать датасет
- предобработать данные: составить список уникальных пользователей и фильмов
- составить матрицу **users - movies** (функция *create_utility_matrix*); значения матрицы - Nan, если нет информации об оценке пользователем фильма, или число, если оценка есть
- составить сингулярное разложение (используя пакетную функцию или собственную реализацию)
- выбрать одного пользователя (один фильм) и найти максимально похожих на него
- вывести рекомендации для пользователя


In [176]:

data = pd.read_csv("ratings.csv")
print(data.head())
data['userId'] = data['userId'].astype('str')
data['movieId'] = data['movieId'].astype('str')
data['ratingId'] = data['rating'].astype('float')



# list of all users
users = data['userId'].unique()


# list of all movies
movies = data['movieId'].unique()

movies= movies.astype('int')
users = users.astype('int')
print()
print(users[:10])
print()
print(movies)
print(len(movies))

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931

[ 1  2  3  4  5  6  7  8  9 10]

[     1      3      6 ... 160836 163937 163981]
9724


In [197]:
def create_utility_matrix(data, formatizer = {'user':0, 'item': 1, 'value': 2}):
    """
        :param data:      Array-like, 2D, nx3
        :param formatizer:pass the formatizer
        :return:          utility matrix (n x m), n=users, m=movies
    """

    itemField = formatizer['item']
    userField = formatizer['user']
    valueField = formatizer['value']

    userList = data.iloc[:,userField].tolist()  #iloc - вытаскивает все значений значения из столбца
    itemList = data.iloc[:,itemField].tolist()
    valueList = data.iloc[:,valueField].tolist()

    users = data.iloc[:,userField].unique().tolist()   # уникальные пользователи 
    items = data.iloc[:,itemField].unique().tolist() # уникальные значения

    users_index =  {users[i]: i for i in range(len(users))}

    pd_dict = {item: [np.nan for i in range(len(users))] for item in items}
    # data = data.drop(index=data.index[0], axis=0, inplace=True)
    for i in range(len(data)):
         item = itemList[i]
         user = userList[i]
         value = valueList[i]
         pd_dict[item][users_index[user]] = value
    #
    X = pd.DataFrame(pd_dict)
    X.index = users

    itemcols = list(X.columns)
    items_index = {itemcols[i]: i for i in range(len(itemcols))}
    #
    # # users_index gives us a mapping of user_id to index of user
    # # items_index provides the same for items
    #
    X = X.replace(np.nan, 0)
    return X, users_index, items_index

user_item_array, users_index, items_index = create_utility_matrix(data)
user_item_matrix = np.matrix(user_item_array)


In [198]:
user_item_matrix[25:30,1:10]

matrix([[0. , 0. , 4. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 3.5, 3. , 3.5, 0. , 0. , 3.5, 0. , 0. ],
        [0. , 0. , 0. , 3.5, 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 5. , 0. , 0. ]])

In [199]:
k = 10 # точность
U = do_svd(user_item_matrix)[0]  
VT = do_svd(user_item_matrix)[2] 



In [208]:
# по похожести фильмов

def recommend_movie_item(liked_movie, VT, output_num=5):
    global rec
    rec = []
    for item in range(len(VT.columns)):
        if item != liked_movie:
            rec.append([item,np.dot(VT[item],VT[liked_movie])]) # перемножаем векторы фильмов
    final_rec = [i[0] for i in sorted(rec, key=lambda x: x[1],reverse=True)] # сортируем в порядке убывания скалярных произведений
    return final_rec[:output_num]

                     
                     # по похожести пользователей

def recommend_movie_user(target_user, U, output_num=100):
    global rec
    users_similarity = []
    for user in range(U.shape[0]):
        if user != target_user:
            users_similarity.append([user, np.dot(U.iloc[user], U.iloc[target_user])])
    sorted_users = [i[0] for i in sorted(users_similarity, key=lambda x: x[1],reverse=True)]
    all_movies = np.array(user_item_array.iloc[sorted_users[0]])
    rec_movies = all_movies.nonzero()
    return rec_movies[0][:output_num]
                     
                     
print(f'Индексы фильмов, похожие на 10 {recommend_movie_item(10, VT)}')
print(f'Индекса фильмов, рекомендованные для 10 пользователя {recommend_movie_user(10, U)}')

Индексы фильмов, похожие на 10 [17, 20, 171, 0, 402]
Индекса фильмов, рекомендованные для 10 пользователя [   5   13   15   20   27   28   34   42   73   75   99  101  102  108
  162  164  179  184  220  232  267  378  446  463  478  486  525  618
  621  642  739  740  752  753  755  759  765  768  770  778  797  806
  809  823  824  885  892  898  899  977  987  991 1002 1012 1015 1022
 1027 1028 1048 1096 1166 1177 1178 1198 1208 1209 1249 1263 1413 1448
 1484 1574 1683 1774 1796 1822 1831 1875 1891 1896 1898 1904 1908 2284
 2310 2320 2397 2419 3141 3188 3327 3355 3646 3647 4510 4908 5913]
